In [ ]:
!python Main_Fed.py -m niid1 -r 30

  0%|                                                     | 0/2 [00:00<?, ?it/s]/home/ahnaf/Fed_Learn/Divergence/data_utils.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)
100%|█████████████████████████████████████████████| 2/2 [00:09<00:00,  4.69s/it]
Local update for all nodes for mode-d2d_clus completed
Average train loss pre-aggregate 0.0132
End of round- Entering aggregation
End of round- Entering aggregation
End of round- Entering aggregation
End of round- Entering aggregation
End of round- Entering aggregation
Aggregation for mode d2d_clus completed
Model_testing completed
Cycle 0 for mode d2d_clus completed
100%|█████████████████████████████████████████████| 2/2 [00:09<00:00,  4.52s/it]
Local update for all nodes for mode-d2d completed
Average train loss pre-aggregate 0.00281


100%|█████████████████████████████████████████████| 2/2 [00:08<00:00,  4.29s/it]
Local update for all nodes for mode-centr_fed completed
Average train loss pre-aggregate 0.00211
End of round- Entering aggregation
Aggregation for mode centr_fed completed
Model_testing completed
Cycle 0 for mode centr_fed completed
100%|████████████████████████████████████████████| 2/2 [06:06<00:00, 183.33s/it]
SGD Training completed-Moving to calculate Divergence
SGD Divergence concluded
100%|█████████████████████████████████████████████| 2/2 [00:08<00:00,  4.47s/it]
Local update for all nodes for mode-d2d_clus completed
Average train loss pre-aggregate 0.00214
End of round- Entering aggregation
End of round- Entering aggregation
End of round- Entering aggregation
End of round- Entering aggregation
End of round- Entering aggregation
Aggregation for mode d2d_clus completed
Model_testing completed
Cycle 1 for mode d2d_clus completed
100%|█████████████████████████████████████████████| 2/2 [00:08<00:00,  4.

100%|█████████████████████████████████████████████| 2/2 [00:08<00:00,  4.38s/it]
Local update for all nodes for mode-centr_fed completed
Average train loss pre-aggregate 1.67e-05
End of round- Entering aggregation
Aggregation for mode centr_fed completed
Model_testing completed
Cycle 1 for mode centr_fed completed
 50%|██████████████████████                      | 1/2 [03:16<03:16, 196.70s/it]